In [1]:
# import modules
import warnings
warnings.filterwarnings('ignore')

import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, log_loss, f1_score

import random
from math import floor
from scipy.stats import mode, scoreatpercentile

print("Seaborn version : ", sns.__version__)
sns.set()
#sns.set_style('whitegrid')
sns.set_color_codes()
sns.set_theme(style="ticks", palette="pastel")

Seaborn version :  0.11.1


In [5]:
# upload data
train = pd.read_csv('/Users/gangtaro/competition_data/DACON/14thMonthlyDacon/open/train.csv',
                   index_col=0)
test = pd.read_csv('/Users/gangtaro/competition_data/DACON/14thMonthlyDacon/open/test.csv',
                  index_col=0)
submit = pd.read_csv('/Users/gangtaro/competition_data/DACON/14thMonthlyDacon/open/sample_submission.csv')

In [6]:
for df in [train, test]:
    # Categorical features with Binary encode (0 or 1; two categories)
    for bin_feature in ['gender', 'car', 'reality', 'work_phone', 'phone', 'email']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    df.drop('FLAG_MOBIL', axis = 1, inplace = True)
    df['adj_DAYS_EMPLOYED_replace_0'] = -df.DAYS_EMPLOYED.replace({365243 : 0})
    df['DAYS_EMPLOYED_missing'] = (df.DAYS_EMPLOYED == 365243).astype('int')
    df['adj_begin_month'] = -df.begin_month
    df['adj_income_type'] = df.income_type
    df.loc[df.income_type == 'Student', 'adj_income_type'] = 'Working'
    df['adj_edu_type'] = df.edu_type 
    df.loc[df.edu_type == 'Academic degree', 'adj_edu_type'] = 'Higher education'
    df['adj_family_type'] = df['family_type']
    df['adj_family_type'].loc[(df.family_type == 'Single / not married')&(df.family_size - df.child_num == 2)] = 'Married'
    df['exp_num'] = 0
    df['exp_num'].loc[df.family_type == 'Married'] = 2
    df['exp_num'].loc[df.family_type == 'Civil marriage'] = 2
    df['exp_num'].loc[df.family_type == 'Separated'] = 1
    df['exp_num'].loc[df.family_type == 'Single / not married'] = 1
    df['exp_num'].loc[df.family_type == 'Widow'] = 1
    df['odd_family_size'] = 0
    df['odd_family_size'].loc[(df.family_size - df.child_num) != df.exp_num] = 1
    df['_single_parents'] = ((df.family_type == 'Single / not married')&(df.child_num != 0)).astype('int')
    df['_single_live'] = (df.family_size == 1).astype('int')
    df['adj_occyp_type'] = df.occyp_type.fillna('missing')
    df['_missing_occyp_type'] = df.occyp_type.isna().astype('int')
    df.loc[(df.DAYS_EMPLOYED == 365243)&(df.occyp_type.isna()), 'adj_occyp_type'] = 'inoccyp'
    df.loc[(df.DAYS_EMPLOYED != 365243)&(df.occyp_type.isna()), 'adj_occyp_type'] = 'non_enter'
    df['_age'] = -df.DAYS_BIRTH/365.25

    df['ID'] = \
    df['gender'].astype('str') + \
    df['car'].astype('str') + \
    df['reality'].astype('str') + '_' + \
    df['child_num'].astype('str') + '_' + \
    df['income_total'].astype('str') + '_' + \
    df['income_type'].astype('str') + '_' + \
    df['family_type'].astype('str') + '_' + \
    df['house_type'].astype('str') + '_' + \
    df['phone'].astype('str') + '_' + \
    df['email'].astype('str') + '_' + \
    df['family_size'].astype('str') + '_' + \
    df['DAYS_BIRTH'].astype('str') + '_' + \
    df['DAYS_EMPLOYED'].astype('str') + '_' + \
    df['occyp_type'].astype('str') 
    
    df['_card_num'] = df.groupby('ID').ID.transform(len)
    
    df['adj_begin_month'] = -df.begin_month
    df['_begin_month_max'] = df.groupby('ID').adj_begin_month.transform(max)
    df['_begin_month_mean'] = df.groupby('ID').adj_begin_month.transform(np.mean)
    df['_begin_month_min'] = df.groupby('ID').adj_begin_month.transform(min)

In [19]:
pd.concat([train.drop(['credit'], axis = 1), test])

,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,...,_single_parents,_single_live,adj_occyp_type,_missing_occyp_type,_age,ID,_card_num,_begin_month_max,_begin_month_mean,_begin_month_min
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,...,0,0,non_enter,1,38.053388,000_0_202500.0_Commercial associate_Married_Mu...,5,58.0,33.200000,6.0
1,0,0,1,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,...,0,0,Laborers,0,31.156742,001_1_247500.0_Commercial associate_Civil marr...,9,13.0,7.444444,3.0
2,1,1,1,0,450000.0,Working,Higher education,Married,House / apartment,-19087,...,0,0,Managers,0,52.257358,111_0_450000.0_Working_Married_House / apartme...,6,48.0,28.666667,7.0
3,0,0,1,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,...,0,0,Sales staff,0,41.308693,001_0_202500.0_Commercial associate_Married_Ho...,2,52.0,44.500000,37.0
4,0,1,1,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,...,0,0,Managers,0,41.169062,011_0_157500.0_State servant_Married_House / a...,1,26.0,26.000000,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36452,1,0,1,0,202500.0,Working,Incomplete higher,Married,House / apartment,-18593,...,0,0,Accountants,0,50.904860,101_0_202500.0_Working_Married_House / apartme...,3,19.0,9.000000,4.0
36453,0,0,1,0,202500.0,Working,Secondary / secondary special,Civil marriage,House / apartment,-10886,...,0,0,Laborers,0,29.804244,001_0_202500.0_Working_Civil marriage_House / ...,2,34.0,34.000000,34.0
36454,1,1,1,0,292500.0,Working,Secondary / secondary special,Married,House / apartment,-21016,...,0,0,Medicine staff,0,57.538672,111_0_292500.0_Working_Married_House / apartme...,5,55.0,23.600000,-0.0


In [21]:
personal_info = train.drop(['credit', 'begin_month'], axis = 1).drop_duplicates(subset="ID", keep='first', inplace=False, ignore_index=True)
personal_info_test = pd.concat([train.drop(['credit'], axis = 1), test]).drop(['begin_month'], axis = 1).drop_duplicates(subset="ID", keep='first', inplace=False, ignore_index=True)

In [ ]:
personal_info = train.drop(['credit', 'begin_month'], axis = 1).drop_duplicates(subset="ID", keep='first', inplace=False, ignore_index=True)

child_fee = (personal_info.income_total/personal_info.family_size)[(personal_info._age > 33) & (personal_info._age < 37) & (personal_info.child_num == 1)].mean()

def child_fee_age_weights(x) : 
    from scipy.stats import norm
    sd = personal_info._age.std()
    return norm(35, scale = sd).pdf(x) / norm(35, scale = sd).pdf(35)



In [10]:
train.info()
features = ['gender', 'car', 'reality', 'child_num', 'income_total', 'adj_income_type', 'adj_edu_type', 
            'adj_family_type', 'house_type', '_age', 'adj_DAYS_EMPLOYED_replace_0', 'DAYS_EMPLOYED_missing',
            'adj_begin_month', 'work_phone', 'phone', 'email', 'adj_occyp_type', '_missing_occyp_type', 'odd_family_size', 
            '_single_parents', '_single_live', 'ID', '_card_num', '_begin_month_max', '_begin_month_mean', '_begin_month_min']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26457 entries, 0 to 26456
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       26457 non-null  int64  
 1   car                          26457 non-null  int64  
 2   reality                      26457 non-null  int64  
 3   child_num                    26457 non-null  int64  
 4   income_total                 26457 non-null  float64
 5   income_type                  26457 non-null  object 
 6   edu_type                     26457 non-null  object 
 7   family_type                  26457 non-null  object 
 8   house_type                   26457 non-null  object 
 9   DAYS_BIRTH                   26457 non-null  int64  
 10  DAYS_EMPLOYED                26457 non-null  int64  
 11  work_phone                   26457 non-null  int64  
 12  phone                        26457 non-null  int64  
 13  email           

In [12]:
train[features].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26457 entries, 0 to 26456
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   gender                       26457 non-null  int64  
 1   car                          26457 non-null  int64  
 2   reality                      26457 non-null  int64  
 3   child_num                    26457 non-null  int64  
 4   income_total                 26457 non-null  float64
 5   adj_income_type              26457 non-null  object 
 6   adj_edu_type                 26457 non-null  object 
 7   adj_family_type              26457 non-null  object 
 8   house_type                   26457 non-null  object 
 9   _age                         26457 non-null  float64
 10  adj_DAYS_EMPLOYED_replace_0  26457 non-null  int64  
 11  DAYS_EMPLOYED_missing        26457 non-null  int64  
 12  adj_begin_month              26457 non-null  float64
 13  work_phone      